In [1]:
from vox_articles_to_db import create_db 

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

from numpy.random import rand, RandomState
from numpy import array, matrix, linalg
from scipy import spatial
import math

from sklearn.decomposition import TruncatedSVD, NMF

#Stemming and Lemmatizing packages
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize   
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer
import Stemmer

In [2]:
english_stemmer = Stemmer.Stemmer('en')
snowball = SnowballStemmer('english')      

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: english_stemmer.stemWords(analyzer(doc))

In [3]:
vox_raw_df = create_db()
vox_df = vox_raw_df[vox_raw_df[1]!='']
vox_df.shape
doc_bodies = vox_df[1]

In [21]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(doc_bodies)
features = vectorizer.get_feature_names()
kmeans = KMeans()
kmeans.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=8, n_init=10,
    n_jobs=1, precompute_distances=True, random_state=None, tol=0.0001,
    verbose=0)

In [33]:
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
print "top features for each cluster:"
for num, centroid in enumerate(top_centroids):
    print "%d: %s" % (num, ", ".join(features[i] for i in centroid))

top features for each cluster:
0: marijuana, legalization, drug, pot, medical, alcohol, dc, colorado, drugs, state
1: percent, tax, students, income, people, college, economy, economic, rate, workers
2: health, obamacare, insurance, care, medicaid, coverage, law, subsidies, states, plans
3: ebola, outbreak, health, virus, disease, africa, liberia, leone, west, sierra
4: sex, marriage, court, supreme, lgbt, couples, marriages, discrimination, gay, rights
5: police, ferguson, officers, black, brown, officer, shooting, wilson, justice, protests
6: people, like, new, just, time, obama, said, clinton, women, trump
7: isis, iran, israel, iraq, obama, russia, nuclear, israeli, deal, syria


In [22]:
def print_topic_descriptions(articles):
    vectorizer = TfidfVectorizer(stop_words='english')
    x = vectorizer.fit_transform(articles)
    features = vectorizer.get_feature_names()
    kmeans = KMeans(n_clusters = 5)
    kmeans.fit(x)
    sub_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
    assigned_cluster = kmeans.transform(x).argmin(axis=1)
    for num, centroid in enumerate(sub_centroids):
        print "%d: %s" % (num, ", ".join(features[i] for i in centroid))
    return assigned_cluster
        

In [38]:
assigned_cluster = kmeans.transform(X).argmin(axis=1)


def sub_topics(articles, category, target_category):
    sub_articles = articles[category==target_category]
    print len(sub_articles)
    assigned_articles = print_topic_descriptions(sub_articles)
    return sub_articles, assigned_articles

sub_articles, article_categories = sub_topics(doc_bodies, assigned_cluster, 5)

371
0: baltimore, gray, police, mosby, injury, freddie, officers, spinal, van, cord
1: wilson, brown, grand, jury, mcculloch, police, evidence, shooting, ferguson, darren
2: police, officers, officer, force, rice, use, garner, killed, gun, shooting
3: police, black, people, officers, white, percent, ferguson, justice, americans, racial
4: police, ferguson, gas, protesters, tear, st, louis, night, protests, brown


In [40]:
sub_articles, article_categories = sub_topics(sub_articles, article_categories, 3)

1


ValueError: n_samples=1 should be >= n_clusters=5

In [25]:
sub_articles, article_categories = sub_topics(sub_articles, article_categories, 1)

41
0: subsidies, fossil, fuel, countries, egypt, government, international, iea, reforms, prices
1: oil, prices, price, saudi, production, opec, shale, falling, states, arabia
2: prices, gasoline, gas, oil, gallon, suvs, driving, efficient, vehicles, consumers
3: pipeline, oil, keystone, sands, xl, alberta, obama, rail, canada, project
4: oil, gas, imports, energy, coal, power, shell, drilling, states, middle
